
---

# 🏆 ESAA 수상작 리뷰 보고서

**주제:** 항공편 연착(Delay) 분류
**제출 모델:** CatBoost 단일 모델(+ class self-weight)
**초기 전략:** CatBoost · XGBoost · TabNet 3모델 → Pseudo Labeling → Semi-Supervised 학습

---

## 1) 한눈에 보는 요약

* **아이디어 핵심**

  * 날짜·시간·공항·항공사 같은 “**특정 값 자체가 의미 있는**” 고카디널리티 범주형 특성에 맞춰, **모든 주요 피처를 범주형(object)로 변환**하고 **CatBoost**로 학습.
  * **결측 복구**를 데이터 관계(항공사↔Carrier_ID, 출발·도착 공항쌍의 평균 비행시간)로 체계적으로 해결.
  * **클래스 불균형**은 **self-weight(class weight)** 로 보정.

* **스코어(작성자가 제공한 수치)**

  * CatBoost 단일 **private 0.6363**
  * Pseudo label 적용 후 **private 0.6605**
  * 단, 작성자는 “pseudo label을 함께 쓰면 성능이 떨어졌다”라고 기술함 → **공개/비공개 보드 혼용 가능성** (아래 ‘검증·평가’에서 정리)

* **제출 결정**: 파라미터 최적화(optimization) 없이, **데이터 최대 활용(=결측 많이 보존)**을 위해 **CatBoost+class weight 단일 모델** 제출

---

## 2) 데이터 전처리 & 특성 공학(Feature Engineering)

### (1) 날짜 결합: Month + Day_of_Month → **Day(1~366)**

* “**특정 기념일/특정 날짜 자체**”가 연착에 영향을 준다는 가설로, **순서성(ordinal)** 을 제거하고 **카테고리(명목형)** 로 사용.

### (2) 항공사 식별자 복구: **Airline ↔ Carrier_ID(DOT)**

* 데이터 설명에 근거해 1:1 대응으로 간주.
* **train**에서 Airline→Carrier_ID 사전을 만든 뒤, 결측인 Carrier_ID를 복구.
* **test**는 복구 불가 행에 대해 **최빈값 대체** + 가능한 항목은 Airline로 복구.

### (3) 불필요/중복 컬럼 제거

* `Month, Day_of_Month`(Day로 대체), `Cancelled, Diverted`(전부 0),
  `Origin_Airport, Destination_Airport`(각 ID와 1:1),
  `Carrier_Code(IATA)`(항공사 N:1), `Origin_State, Destination_State`(공항ID가 더 정보력 높음).

### (4) 출/도착 **예상시각(EDT/EAT)** 복구

* HHMM → **분 단위(0~1440)** 로 변환, 시차 이슈는 **mod 1440** 로 해결.
* **(Origin_Airport_ID, Destination_Airport_ID) 쌍별 평균 비행시간**을 구해 결측을 상호 복구.
* **test**에서 둘 다 결측이면 각 컬럼 **최빈값**으로 채움.

### (5) CatBoost 친화적 **범주화**

* EDT/EAT: **30분 단위 48bin** → **범주형**(`EDT`, `EAT`).
* Distance: 100단위 **50bin** → **범주형**.
* `EDT, EAT, Distance, Origin_Airport_ID, Destination_Airport_ID, Carrier_ID(DOT)` **전부 object**로 캐스팅.

> **전처리의 미덕**:
> 순서·거리 개념을 없애고 “**같은 구간/같은 공항/같은 항공사면 유사한 패턴**”이라는 **명목형 가정**을 CatBoost가 잘 소화하게 설계.

---

## 3) 모델링 & 학습

### (1) 불균형 보정: **self-weight(class_weights)**

* `counts = train_y.value_counts()`
* `class_weight = [minority 비율, majority 비율]` 형태로 계산해 CatBoost에 투입
* (참고) `auto_class_weights="Balanced"` 와 유사하게 동작

### (2) CatBoost 설정

* `CatBoostClassifier(random_seed=42, cat_features=[0..7], class_weights=class_weight, verbose=0)`
* **최적화(Optimization) 미실시**:

  * 결측이 많아 **검증셋 분할 시 학습에 투입 가능한 표본이 줄어드는 손실**을 회피
  * Public 기준으로 **미튜닝 모델이 오히려 약간 우세** → 단일 모델 제출 결정

### (3) 추론/제출

* `predict_proba(test_x)` → 제출 스키마에 맞춰 proba 저장

---

## 4) 검증·평가 관점에서의 메모

* **Pseudo Label 성능 서술의 불일치**

  * 본문: “pseudo label을 쓰면 성능이 떨어졌다”
  * 수치 예: CatBoost 0.6363 → Pseudo 0.6605(**상승**)
  * 추정: **Public과 Private, 혹은 실험 간 스플릿 차이**로 인한 서술 혼선 가능성
  * **권장:** 보고서 본문엔 **“공개/비공개 리더보드 기준과 실험 로그를 명시”** 해 혼선을 제거

* **검증 전략의 트레이드오프**

  * **장점:** 데이터 최대 활용(결측 복구+학습 표본 극대화)
  * **리스크:** 튜닝 부재로 **과/과소적합 탐지 어려움**, **재현 실험의 신뢰구간**이 넓어질 수 있음
  * **보완책:** StratifiedKFold(예: 5-fold)로 **경량 탐색(러프 그리드/랜덤서치)** → 최적화 범위만 좁게라도 확인

---

## 5) 강점(What went well)

1. **문제 특성에 맞춘 명확한 데이터 철학**

   * “값 자체의 식별성이 크다”는 가정 하에 **전면 카테고리화** → CatBoost 시너지 극대화
2. **결측 복구의 데이터-기반성**

   * Airline↔Carrier_ID, 공항쌍 평균 비행시간 등 **도메인 합리성 + 통계 일관성**
3. **실용적 불균형 처치**

   * class weight로 간단/견고하게 균형 확보
4. **파이프라인 깔끔함·재현성**

   * seed 고정, 처리 순서 명확, 제출 스키마 일치

---

## 6) 아쉬운 점(Next step)

1. **검증 설계의 부족**

   * 최소한의 Fold-CV로 **튜닝 영향의 방향성**만이라도 확인 권장
   * Public/Private 간 **분산 추정(표준오차, IQR)** 기록하면 의사결정 설득력↑
2. **Pseudo Label 실험 정리**

   * **교차검증 기반 교사 모델** → 신뢰구간 내에서만 필터링(예: pmax-psecond ≥ τ)
   * **Soft pseudo label**(proba) vs **Hard label** 비교 표 삽입
3. **외생 변수 확장(추가 피처)**

   * **공휴일/성수기 플래그**, **공항·항로 혼잡도(출발·도착 동일 시각대 트래픽)**
   * **기상 요약(맑음/비/눈, 풍속 구간)** — *데이터 접근 가능 시*
4. **해석 가능성 보강**

   * CatBoost의 `get_feature_importance`(PredictionValuesChange) + **SHAP summary**로
     *“어느 시간대/공항쌍/항공사가 Delay 확률을 끌어올렸는가”* 시각화

---

## 7) 재현성 체크리스트

* [ ] Seed(42) 고정 재확인
* [ ] Train/Test 동일 전처리(특히 **EDT/EAT binning**·**Distance binning**)
* [ ] Airline→Carrier_ID 사전은 **train에서만** 생성, test는 **사전·최빈값**으로 처리
* [ ] (OAID, DAID) 평균 비행시간 계산 시 **train만 사용**, test 누수 방지
* [ ] Class weight 계산 로직과 **라벨 인덱스 순서** 일치 확인
* [ ] 제출 컬럼 순서/인덱스(sample_submission와 동일) 검증

---

